In [196]:
import polars as pl
import pandas as pd
import json
from pprint import pprint
from chromadb.utils import embedding_functions

# import
from langchain_community.document_loaders import DirectoryLoader, JSONLoader
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings
)
from langchain_community.embeddings import OpenAIEmbeddings, AzureOpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import CharacterTextSplitter
from openai import OpenAI, AzureOpenAI
from langchain_openai import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser


In [139]:
config = json.load(open('../config.json'))

organization = ""

api_source = 'OpenAI'

api_key = config[api_source]['openai_api_key'] #constants.AZURE_OPENAI_KEY
api_endpoint = config[api_source]['openai_api_endpoint']  

In [110]:
label_dat = pl.read_csv('../data/labels_w_ADRs.csv')
label_dat.head()

file,set_id,spl_version,title,ingredient_rx_cui,ingredient_name,pt_meddra_id,pt_meddra_term,cohort_id,condition_name,positive_controls,label,cohort_id_right,drug_concept_id,affect,reference
str,str,i64,str,i64,str,i64,str,i64,str,i64,i64,i64,i64,i64,i64
"""../data/2024_l…","""4e8516dc-a3c6-…",1,"""These highligh…",6064,"""isotretinoin""",10019851,"""hepatotoxicity…",500000301,"""acute liver in…",28,1,null,null,null,null
"""../data/2024_l…","""0e98593a-8424-…",9,"""These highligh…",20352,"""carvedilol""",10017955,"""gastrointestin…",500001001,"""gi bleed""",21,1,null,null,null,null
"""../data/2024_l…","""5c8cebcd-699f-…",100,"""These highligh…",8754,"""propafenone""",10060795,"""hepatic enzyme…",500000301,"""acute liver in…",14,1,null,null,null,null
"""../data/2024_l…","""ab8616a5-ea72-…",10,"""These highligh…",40254,"""valproate""",10018830,"""haematemesis""",500001001,"""gi bleed""",11,1,null,null,null,null
"""../data/2024_l…","""10a3fbc9-00c6-…",1,"""These highligh…",7646,"""omeprazole""",10019692,"""hepatic necros…",500000301,"""acute liver in…",15,1,null,null,null,null


In [111]:
len(label_dat['file'].unique())

860

In [112]:
loader = JSONLoader(
    file_path='../data/2024_latest_labels/latest_labels_ingredients/20240111_6370e342-16a5-4339-a13f-ba275fd5de09.json',
    jq_schema='.sections',
    text_content=False)

data = loader.load()
pprint(data)

[Document(page_content='{"AR": "ADVERSE REACTIONS\\n\\n\\nClinical Trial Adverse Events. Clomiphene citrate, at recommended dosages, is generally well tolerated. Adverse reactions usually have been mild and transient and most have disappeared promptly after treatment has been discontinued. Adverse experiences reported in patients treated with clomiphene citrate during clinical studies are shown in Table 2.\\n\\nTable 2. Incidence of Adverse Events in Clinical Studies (Events Greater than 1%) (n = 8029*)\\n\\n\\n\\n\\n\\n\\n\\u00a0*Includes 498 patients whose reports may have been duplicated in the event totals and could not be distinguished as such. Also, excludes 47 patients who did not report symptom data.\\n\\n\\n\\n\\n\\nAdverse Event\\n\\n\\n%\\n\\n\\n\\n\\nOvarian Enlargement\\n\\n\\n13.6\\n\\n\\n\\n\\nVasomotor Flushes\\n\\n\\n10.4\\n\\n\\n\\n\\nAbdominal-Pelvic Discomfort/Distention/Bloating\\n\\n\\n5.5\\n\\n\\n\\n\\nNausea and Vomiting\\n\\n\\n2.2\\n\\n\\n\\n\\nBreast Discomfo

In [165]:
len(label_dat['file'].unique())

860

In [181]:
label_dat.head()

file,set_id,spl_version,title,ingredient_rx_cui,ingredient_name,pt_meddra_id,pt_meddra_term,cohort_id,condition_name,positive_controls,label,cohort_id_right,drug_concept_id,affect,reference
str,str,i64,str,i64,str,i64,str,i64,str,i64,i64,i64,i64,i64,i64
"""../data/2024_l…","""4e8516dc-a3c6-…",1,"""These highligh…",6064,"""isotretinoin""",10019851,"""hepatotoxicity…",500000301,"""acute liver in…",28,1,null,null,null,null
"""../data/2024_l…","""0e98593a-8424-…",9,"""These highligh…",20352,"""carvedilol""",10017955,"""gastrointestin…",500001001,"""gi bleed""",21,1,null,null,null,null
"""../data/2024_l…","""5c8cebcd-699f-…",100,"""These highligh…",8754,"""propafenone""",10060795,"""hepatic enzyme…",500000301,"""acute liver in…",14,1,null,null,null,null
"""../data/2024_l…","""ab8616a5-ea72-…",10,"""These highligh…",40254,"""valproate""",10018830,"""haematemesis""",500001001,"""gi bleed""",11,1,null,null,null,null
"""../data/2024_l…","""10a3fbc9-00c6-…",1,"""These highligh…",7646,"""omeprazole""",10019692,"""hepatic necros…",500000301,"""acute liver in…",15,1,null,null,null,null


In [223]:
records = []

In [225]:
records

['5 WARNINGS AND PRECAUTIONS\n\n\n\n\n\nNeurological Disorders of the Eye:  Interrupt Unituxin for dilated pupil with sluggish light reflex or other visual disturbances and permanently discontinue Unituxin for recurrent eye disorders or loss of vision. (5.2)\nProlonged Urinary Retention and Transverse Myelitis: Permanently discontinue Unituxin and institute supportive care. (5.2)\nReversible Posterior Leukoencephalopathy Syndrome (RPLS): Permanently discontinue Unituxin and institute supportive care for signs and symptoms of RPLS. (5.2)\nCapillary Leak Syndrome and Hypotension:  Administer required prehydration and monitor patients closely during treatment.  Depending upon severity, manage by interruption, infusion rate reduction, or permanent discontinuation. (5.3, 5.4)\nInfection:  Interrupt until resolution of systemic infection. (5.5)\nBone Marrow Suppression:  Monitor peripheral blood counts during Unituxin therapy. (5.6)\nElectrolyte Abnormalities:  Monitor serum electrolytes clo

In [211]:
def metadata_func(record: dict, metadata: dict) -> dict:
    records.append(record)
    # metadata["sections"] = record.get("sections")

    return metadata


In [227]:
'.users[] | "\(.first) \(.last)"'
'.sections | "\(.WP) \(.BW)"'
documents = []
for file in label_dat['file'].unique():
    loader = JSONLoader(file_path=file,
                        jq_schema='.sections | "\(.WP) \(.BW) \(.WA) \(.PR)"',
                        text_content=False)
    documents.extend(loader.load())

text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=20, separator=" ", add_start_index=True)
chunks = text_splitter.split_documents(documents)

Created a chunk of size 507, which is longer than the specified 500
Created a chunk of size 1099, which is longer than the specified 500


In [230]:
chunks[100].metadata

{'source': '/Users/undinagisladottir/Documents/Columbia/Tatonetti_Lab/causal-drug-ades/data/2024_latest_labels/latest_labels_ingredients/20240217_413c645e-074d-4276-b39a-d83521077602.json',
 'seq_num': 1,
 'start_index': 3850}

In [236]:
len(chunks)

20644

: 

In [231]:
chunks[100].page_content

'was reported in patients receiving abiraterone acetate in combination with prednisone, following interruption of daily steroids and/or with concurrent infection or stress.\nMonitor patients for symptoms and signs of adrenocortical insufficiency, particularly if patients are withdrawn from prednisone, have prednisone dose reductions, or experience unusual stress.\nSymptoms and signs of adrenocortical insufficiency may be masked by adverse reactions associated with mineralocorticoid excess seen in'

In [232]:
db = Chroma.from_documents(chunks, OpenAIEmbeddings(openai_api_key=api_key), persist_directory="../../WP_Chroma")

'{"AR": "6 ADVERSE REACTIONS \\n\\nThe following important adverse reactions are described below and elsewhere in the labeling:\\n\\n\\n\\u2022Diabetic Ketoacidosis in Patients with Type 1 Diabetes Mellitus and Other Ketoacidosis [see Warnings and Precautions (5.1)]\\n\\n\\n\\u2022Volume Depletion [see Warnings and Precautions (5.2)]\\n\\n\\n\\u2022Urosepsis and Pyelonephritis [see Warnings and Precautions (5.3)]\\n\\n\\n\\u2022Hypoglycemia with Concomitant Use with Insulin and Insulin Secretagogues [see Warnings'

In [93]:
db = Chroma(persist_directory="../../Chroma", embedding_function=OpenAIEmbeddings(api_key=config[api_source]['openai_api_key']))
retriever = db.as_retriever(search_kwargs={"k": 3})

print(len(db.get()['documents']))

7398


In [101]:
retriever

AttributeError: 'VectorStoreRetriever' object has no attribute 'search'

In [104]:
len(chunks[0].page_content)

9794

## Run rest on EIR

In [44]:
retriever = db.as_retriever()

In [45]:
from langchain.prompts import ChatPromptTemplate

template = """
You are an expert in summarizing current information. Create a concise and comprehensive summary
of the known knowledge for the causal assocation between a drug and an adverse event. 
Craft a summary that is detailed, thorough, in-depth, and complex, while maintaining clarity and 
conciseness. Incorporate the retrieved context, and previous medical literature.
If you don't know the answer just say I don't know.
Keep your summarize to less than 3 sentences.
Question: {question} 
Context: {context} 
Answer:
"""
prompt = ChatPromptTemplate.from_template(template)

print(prompt)

input_variables=['context', 'question'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="\nYou are an expert in summarizing current information. Create a concise and comprehensive summary\nof the known knowledge for the causal assocation between a drug and an adverse event. \nCraft a summary that is detailed, thorough, in-depth, and complex, while maintaining clarity and \nconciseness. Incorporate the retrieved context, and previous medical literature.\nIf you don't know the answer just say I don't know.\nKeep your summarize to less than 3 sentences.\nQuestion: {question} \nContext: {context} \nAnswer:\n"))]


In [51]:
llm = ChatOpenAI(api_key=api_key, model_name="gpt-4-1106-preview", temperature=0)

rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()} 
    | prompt 
    | llm
    | StrOutputParser() 
)

query = "What is the evidence, if any, supporting the causal association between lisinopril and acute kidney injury?"
rag_chain.invoke(query)

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-DxK9C***************************************2DQN. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [52]:
from typing import List, Optional

from vertexai.language_models import TextEmbeddingInput, TextEmbeddingModel

In [53]:
def text_embedding(words) -> list:
    """Text embedding with a Large Language Model."""
    model = TextEmbeddingModel.from_pretrained("textembedding-gecko@001")
    embeddings = model.get_embeddings(words)
    for embedding in embeddings:
        vector = embedding.values
        print(f"Embedding Vectors: {vector}")
        print(f"Length of Embedding Vector: {len(vector)}")
    return vector


In [67]:
text_embedding(["Hello", "World"])

GoogleAuthError: 
Unable to authenticate your request.
Depending on your runtime environment, you can complete authentication by:
- if in local JupyterLab instance: `!gcloud auth login` 
- if in Colab:
    -`from google.colab import auth`
    -`auth.authenticate_user()`
- if in service account or other: please follow guidance in https://cloud.google.com/docs/authentication

In [72]:
import json

In [85]:
f = open('../results/support_basic_embed.json')
data = json.load(f)
print(data.keys())


dict_keys(['error'])
